这个例子比较简单，利用 knn 算法给手写体分类，分成 0 - 9 的数字。

训练集和测试集分部都很有规律。首先文件名以 num_order 方式命名，num 代表这个是什么数字，order 代表是类别为 num 的第 order 个样例。

而一个文件内部用 0 1 数字模拟像素点构成一个 32 X 32 的图像。

在这个例子里，吧 32 X 32 flatten 了。

In [1]:
import os
import numpy as np
from operator import itemgetter

In [2]:
x = np.zeros((3,3))
y = np.ones((1,3))
np.append(x, y, axis = 0)

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [1., 1., 1.]])

In [3]:

# flatten
def img_to_vector(filename):
    with open(filename, 'r') as f:
#         img_vec = np.zeros((1, 32 * 32))
        img_vec = np.array([])
        lines   = f.readlines()
#         print(len(lines)) # 32
#         print(len(lines[0])) # 33 多了一个 \n
        for line in lines:
            img_vec = np.append(img_vec, list(map(int, line[0:32])))
        return img_vec

# 归一化不用做了，因为值只有 0,1 不存在某个值过大影响距离权重
def pre_handle():
    list_train_dirs = os.listdir('./trainingDigits')
    
    # shape = (m, 1024) m 为例子个数
    train_sample_num      = len(list_train_dirs)
    x_train               = np.zeros((train_sample_num, 1024))
    y_train               = np.zeros((train_sample_num))
    
    i = 0
    for train_file in list_train_dirs:
        vec           = img_to_vector('./trainingDigits' + '/' + train_file)
        x_train[i, :] = vec
        filename, extension = os.path.splitext(train_file)
        class_name    = filename.split('_')[0] # 0_0
        y_train[i]    = int(class_name)
        i            += 1
    # 准备测试集
    i = 0
    list_test_dirs  = os.listdir('./testDigits')
    test_sample_num = len(list_test_dirs)
    x_test          = np.zeros((test_sample_num, 1024))
    y_test          = np.zeros((test_sample_num))
    for test_file in list_test_dirs:
        vec           = img_to_vector('./testDigits' + '/' + test_file)
        x_test[i, :]  = vec
        filename, extension = os.path.splitext(test_file)
        class_name    = filename.split('_')[0] # 0_0
        y_test[i]     = int(class_name)
        i            += 1
    return x_train, y_train, x_test, y_test
            
x_train, y_train, x_test, y_test = pre_handle()


In [4]:
def knn(sample, x_train, y_train, k):
    sq        = (x_train - sample) ** 2
    distances = (sq.sum(axis = 1)) ** 0.5 # 为什么是 1？保留每个计算的距离
    # 按照距离从小到大排序
    sort_index = distances.argsort()
    # 求前 k 个最小距离中，类别最多的
    class_count = {}
    for i in range(k):
        index = sort_index[i]
        label = y_train[index]
        class_count[label] = class_count.get(label, 0) + 1
    # 用 dict 的 key-value 元组根据 value 逆序排序
    res = sorted(class_count.items(), key = itemgetter(1), reverse = True)
#     print(res)
    return res[0][0] # 最多的类别
        
def hand_write(x_train, y_train, x_test, y_test, k):
    x_test_len = len(x_test)
    correct    = 0
    for i in range(x_test_len):
        pre_class = knn(x_test[i], x_train, y_train, k)
#         print(pre_class, y_test[i])
        if pre_class == y_test[i]:
            correct += 1
    print(correct)
    return correct / x_test_len

print(hand_write(x_train, y_train, x_test, y_test, 3))

936
0.9894291754756871


In [5]:
np.array([23,1,222]).argsort()

array([1, 0, 2], dtype=int64)

不得不说正确率是很高的。但是它的问题就是效率实在太低，每一个要预测的数据放进去，就要和模型内已有的数据做运算，只要模型中数据庞大起来，它的运算就要久。

900 多个待预测样本，每个样本要计算 1000+ 次，这样就是 1000 * 900 次计算。